In [1]:
import folium
import pandas as pd

In [3]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [5]:
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

In [7]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [9]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [11]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [13]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

for index, site in launch_sites_df.iterrows():
    coordinate = [site['Lat'], site['Long']]
    # Add a circle for the launch site
    folium.Circle(
        coordinate,
        radius=1000,
        color='#000000',
        fill=True
    ).add_child(folium.Popup(site['Launch Site'])).add_to(site_map)
    
    folium.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site['Launch Site']
        )
    ).add_to(site_map)

site_map

In [15]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch, add a Marker to the map
for index, launch in spacex_df.iterrows():
    coordinate = [launch['Lat'], launch['Long']]
    outcome = launch['class']
    
    # Determine marker color based on the outcome
    color = 'green' if outcome == 1 else 'red'
    
    # Add a marker for the launch site with the outcome
    folium.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:{};"><b>{}</b></div>'.format(color, launch['Launch Site'])
        )
    ).add_to(site_map)

# Display the map
site_map

In [17]:
spacex_df.tail()

,Launch Site,Lat,Long,class
51,CCAFS SLC-40,28.563197,-80.57682,0
52,CCAFS SLC-40,28.563197,-80.57682,0
53,CCAFS SLC-40,28.563197,-80.57682,0
54,CCAFS SLC-40,28.563197,-80.57682,1
55,CCAFS SLC-40,28.563197,-80.57682,0


In [19]:
marker_cluster = MarkerCluster()

In [21]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [23]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

marker_cluster = MarkerCluster().add_to(site_map)

# For each row in spacex_df data frame, create and add a Marker to the marker_cluster
for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"Launch Site: {record['Launch Site']}<br>Class: {record['class']}"
    )
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)
site_map

In [25]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [27]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [29]:
launch_site_coord = (28.5623, -80.5773)  # Replace with actual launch site coordinates
coastline_coord = (28.5623, -80.5678)  # Replace with actual closest coastline coordinates

distance_coastline = calculate_distance(launch_site_coord[0], launch_site_coord[1], coastline_coord[0], coastline_coord[1])
print(f"Distance to coastline: {distance_coastline:.2f} km")

site_map = folium.Map(location=launch_site_coord, zoom_start=10)

folium.Marker(
    location=launch_site_coord,
    popup='Launch Site',
    icon=folium.Icon(color='blue')
).add_to(site_map)

folium.Marker(
    location=coastline_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline)
    )
).add_to(site_map)

coordinates = [launch_site_coord, coastline_coord]
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue')
site_map.add_child(lines)

site_map

Distance to coastline: 0.93 km
